In [1]:
# Import the yfinance. If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf
import pandas as pd
import numpy as np

In [60]:
# get daily, yearly or hourly finance data
stock_data = yf.download(tickers="NVDA", period="1d", interval="1m")

[*********************100%***********************]  1 of 1 completed


In [61]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-05-29 09:30:00-04:00,1130.545044,1133.269897,1128.079956,1129.359985,1129.359985,3738953
2024-05-29 09:31:00-04:00,1133.339966,1133.749878,1127.150024,1127.150024,1127.150024,577984
2024-05-29 09:32:00-04:00,1127.150024,1129.000000,1124.229980,1127.455566,1127.455566,451668
2024-05-29 09:33:00-04:00,1127.435059,1130.158081,1123.248047,1125.099976,1125.099976,412232
2024-05-29 09:34:00-04:00,1125.359985,1127.479980,1123.000000,1126.954956,1126.954956,323838
...,...,...,...,...,...,...
2024-05-29 15:55:00-04:00,1143.099976,1145.989990,1143.000000,1145.739990,1145.739990,196303
2024-05-29 15:56:00-04:00,1145.810059,1146.369995,1145.050049,1145.688843,1145.688843,160726
2024-05-29 15:57:00-04:00,1145.569946,1146.410034,1145.000000,1146.270020,1146.270020,179954


In [72]:
# data.groupby(pd.Grouper(freq='15Min')).size()
df = stock_data.reset_index().groupby(
    pd.Grouper(freq='15Min', key="Datetime")
).agg(
    open=("Open", "first"), 
    close=("Close", "last"),
    low=("Low", "min"),
    high=("High", "max"),
)

In [77]:
df["return"] = df["close"] - df["open"]
df.reset_index().head()


,Datetime,open,close,low,high,return
0,2024-05-29 09:30:00-04:00,1130.545044,1112.262451,1109.020142,1133.749878,-18.282593
1,2024-05-29 09:45:00-04:00,1112.239990,1124.895020,1110.959961,1131.239990,12.655029
2,2024-05-29 10:00:00-04:00,1124.942505,1129.740356,1120.040039,1131.170044,4.797852
3,2024-05-29 10:15:00-04:00,1130.250000,1136.523804,1130.035034,1139.489990,6.273804
4,2024-05-29 10:30:00-04:00,1136.780029,1142.071167,1136.689941,1147.409912,5.291138


In [91]:
# let's plot
import altair as alt
from vega_datasets import data

# source = data.ohlc()
# display(source.head())
source = df.reset_index()

open_close_color = alt.condition(
    "datum.open <= datum.close",
    alt.value("#06982d"),
    alt.value("#ae1325")
)

base = alt.Chart(source).encode(
    alt.X('Datetime:T')
        .axis(format='%H:%M', labelAngle=-80)
        .title('date-time'),
    color=open_close_color
)

rule = base.mark_rule(size=1).encode(
    alt.Y('low:Q')
        .title('Price')
        .scale(zero=False),
    alt.Y2('high:Q')
)

bar = base.mark_bar(size=10).encode(
    alt.Y('open:Q'),
    alt.Y2('close:Q')
)

(rule+bar).properties(
    width=800,
    height=300
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

alt.LayerChart(...)